In [1]:
def generate_5param_STC(datasheet_values):
    """
    
    This function will return the 5 parameters of a single diode model of the solar cell given datasheet values
    
    Input  : Datasheet Parameters: Voc, Isc, Vm, Im, Ncell at "STC"
             
    Output : Returns the single diode parameter tuple (Rs, Rsh, Isat, Iph, eta)
    
    
    """
    Voc, Isc, Vm, Im, Ncell = datasheet_values
    
    Temp = 25
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Vt = kb*(Temp+273)/q
    
    #Initial Guess Calculation (eta, Rs, Rsh)
    etai = (2*Vm-Voc) / (Ncell*Vt*(np.log((Isc-Im)/Isc)+Im/(Isc-Im)))
    Rsi = Vm/Im - (2*Vm-Voc) / ((Isc - Im) * (np.log((Isc-Im)/Isc) + Im/(Isc-Im)))
    if (Rsi<0):
        Rsi = 0
        num = 1 + (1-Im/Isc)*(np.log(1-Im/Isc))/(1-Vm/Voc)
        den = Im/Vm + (Isc-Im)*np.log(1-Im/Isc)/(Voc-Vm)
        Rshi = num/den
    else :
        Rshi = np.sqrt(Rsi/((Isc/(etai*Ncell*Vt)*np.exp((Rsi*Isc - Voc)/(etai*Ncell*Vt)))))

    # Solving them to get these 3 params
    def para(X):
        eta, Rs, Rsh = X
        f = [Im - Isc + (Vm + Im*Rs - Isc*Rs)/Rsh + (Isc - (Voc - Isc)/Rsh)*np.exp((-Voc + Vm+Im*Rs)/(eta*Ncell*Vt)), Im + Vm*((-((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh))*(np.exp((-Voc + Vm+Im*Rs)/(eta*Ncell*Vt)))-(1/Rsh))/(1+((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh)*Rs*np.exp((-Voc+Vm+Im*Rs)/(eta*Ncell*Vt)))+Rs/Rsh)), 1/Rsh + ((-((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh))*(np.exp((-Voc + Isc*Rs)/(eta*Ncell*Vt)))-(1/Rsh))/(1+((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh)*Rs*np.exp((-Voc+Isc*Rs)/(eta*Ncell*Vt)))+Rs/Rsh))]
        return f

    param_sol = root(para,[etai,Rsi,Rshi])
    eta = param_sol.x[0]
    Rs = param_sol.x[1]
    Rsh = param_sol.x[2]
    if (Rs<0):
        Rs=0
    # Getting the remaining 2 parameters
    Isat = (Isc - (Voc - Isc*Rs)/Rsh)*(np.exp(-Voc/(eta*Ncell*Vt)));
    Iph = Isat * np.exp(Voc / (eta*Ncell*Vt)) + (Voc/Rsh);

    parameters = (Rs, Rsh, Isat, Iph, eta)
    
    return (parameters)
#print (generate_5param_STC((45.9, 9.25, 37.2, 8.76, 72)))

In [2]:
def translate(param, G, T, Isc_NOTC, Isc_STC=None, alpha=None):
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Eg = 1.1
    Rs, Rsh, Isat, Iph, eta = param
    
    ####################### Very important ####################3
    #Isc_STC = Iph           # If module has been degraded in Isc=Iph will work, otherwise translation occur according to Isc_nameplate
    if not Isc_STC:
        Isc_STC = Iph
####################################################################
    
    Rs_n = Rs                    # No change in series resistance
    if not alpha:                # If alpha not given
        print ('Making own alpha')
        alpha = Isc_STC*0.05/100
    
    K = np.log((Isc_NOTC-alpha*(45-25))/Isc_STC)/np.log(800/1000) # (T_NOTC - T_STC), (G_NOTC/G_STC)
    Isc_n = ((G/1000)**K) * (Isc_STC + alpha*(T-25))  # New Isc (Used later for Iph calc) 
    Rsh_n = (1000/G)*Rsh
    Iph_n = Isc_n*(1+Rs_n/Rsh_n)
    Isat_n = Isat*(((T+273)/(25+273))**3)*np.exp((Eg*q)/(eta*kb) * (1/(273+25) - 1/(T+273)))
    return (Rs_n, Rsh_n, Isat_n, Iph_n, eta)

In [3]:
def correction(method, data, datasheet_values, Irrad, Temp, Tcoeff):
    Voc_Nameplate, Isc_Nameplate, Vmp_Nameplate, Imp_Nameplate, _ = datasheet_values
#     print ('Correction going on')
#     print ('Isc_Nameplate', Isc_Nameplate)
    Tm, Gm = Temp, Irrad
    alpha, beta, gamma = Tcoeff
    V_measured, I_measured = data['x'], data['y']
    deltaT = 25-Tm
    
    if (method==1):    
        Isc_measured=I_measured[0]
        VSTC=np.round(V_measured+((beta/100)*(Voc_Nameplate)*deltaT), 2);
        ISTC=np.round(I_measured+(Isc_measured*((1000/Gm)-1))+((alpha/100)*(Isc_Nameplate)*deltaT), 2);
        PSTC=VSTC*ISTC
        data = dict(x=VSTC, y=ISTC)
        id_maxP = PSTC.argmax()
        Imp, Vmp, Pmax = np.round(ISTC[id_maxP],2), np.round(VSTC[id_maxP],2), np.round(PSTC[id_maxP],2)
        # Need to calculate the Voc, Isc also 
        #Extraploating V and I to get Voc and Isc at STC
        datasize = len(V_measured)    
        start = datasize//10

        coeff_STC_V = np.polyfit(ISTC[-start:-1], VSTC[-start:-1], 1)      # (Fitting I = mV + c)
        Voc_STC= np.round(coeff_STC_V[1], 2)

        coeff_STC_I = np.polyfit(VSTC[0:5], ISTC[0:5], 1)      # (Fitting I = mV + c)
        Isc_STC = np.round(coeff_STC_I[1], 2)

        shunt_STC = np.round(-1/coeff_STC_I[0], 2);
        series_STC = np.round(-coeff_STC_V[0], 2); 
        
        FF = np.round(Pmax/(Voc_STC*Isc_STC),2)
        metadata = dict(Voc=[Voc_STC], Isc=[Isc_STC], Vmp=[Vmp], Imp=[Imp], 
                        Pmax=[np.round(Pmax,2)], FF=[FF], Rsh=[shunt_STC], Rs=[series_STC])
    return (data, metadata)

In [4]:
import pandas as pd

def generate_IV(module_param, Temp, datapoints, Ncell, Voc=None, Isc=None):
    """
    This function will generate I-V data points given all the 5 diode parameters 
    
    Input  : 5 parameters: Rs, Rsh, Isat, Iph, eta
             Voc, Temperature(Celcius), N(Number of data points), Ncell
             
    Output : Returns a dictionary of I-V data (size = N) and metadata (data, metadata)
    
    This function will spit out I-V data points. 
    
    ** This auto generates Voc and Isc also if not provided **
    **Temperature is just taken to calculate Vt**
    
    """
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Vt = kb*(Temp+273)/q
    
    Rs, Rsh, Isat, Iph, eta = module_param
    
    # If the Voc argument is not passed explicitly, this will calculate Voc also from the 5 parameters
    if not Voc:             # If Voc=None
        Voc_implicit_eqn = lambda voc: Iph - Isat*(np.exp((voc) / (eta*Ncell*Vt))-1) - (voc)/Rsh # i=0 @Voc
        Vocr = root(Voc_implicit_eqn,50)
        Voc = np.round(Vocr.x[0], 2)
        #print ('Auto generated Voc is :', Voc)
    
    # If the Isc argument is not passed explicitly, this will calculate Isc also from the 5 parameters
    if not Isc:             # If Voc=None
        Isc_implicit_eqn = lambda isc: -isc + Iph - Isat*(np.exp((isc*Rs) / (eta*Ncell*Vt))-1) - (isc*Rs)/Rsh # v=0 @Isc
        Isc = np.round(root(Isc_implicit_eqn,50).x[0], 2)
        #print ('Auto generated Isc is :', Isc)    
    
    # Now solving the implicit equation of the solar cell 
    I_implicit_equn = lambda i: -i + Iph - Isat*(np.exp((v+i*Rs) / (eta*Ncell*Vt))-1) - (v+i*Rs)/Rsh
    V = np.linspace(0,Voc,datapoints)  #Generating the Voltage vector
    I = []
    for v in V:                        # For each value of Voltage, current is calculated
        ior = root(I_implicit_equn,10)
        io = ior.x[0]
        I.append(io)
    
    I = np.round(I, 2)                 # Rounding upto 2 places
    V = np.round(V, 2)
    
    ## Now calculating Pmax, FF, Im, Vm
    df = pd.DataFrame(dict(V=V, I=I))
    df['P'] = df['I']*df['V']
    values = df.loc[df['P'].idxmax()]
    Vm, Im, Pmax = values
    FF = np.round(Pmax/(Voc*Isc),2)
    
    #metadata = dict(Voc=Voc, Isc=Isc, Pmax=np.round(Pmax,2), Vmp=Vm, Imp=Im, FF=FF) 
    metadata = dict(Voc=[Voc], Isc=[Isc], Pmax=[np.round(Pmax,2)], Vmp=[Vm], Imp=[Im], FF=[FF]) 
    data = dict(x=V, y=I)
    
    return (data, metadata)              # Returning a dictionary of data and metadata

In [5]:
from bokeh.models import Slider, CustomJS, Div
from bokeh.models.widgets import Button, TextInput, DataTable, TableColumn
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import Figure, output_notebook, show, ColumnDataSource, output_file
import pandas as pd
from scipy.optimize import root
import numpy as np
from os.path import dirname, join

output_notebook()

def plot_iv(doc):

    datasheet_values = (45.9, 9.25, 37.2, 8.76, 72)        # datasheet_values = Voc, Isc, Vm, Im, Ncell
    Isc_NOTC = 7.47
    alpha, beta, gamma = 0.065, -0.36, -0.5
    datasize = 100
    diode_param = generate_5param_STC(datasheet_values)   # parameters = (Rs, Rsh, Isat, Iph, eta)
    data_pair_STC = generate_IV(diode_param, 25, datasize, 72)
    source_STC = ColumnDataSource(data=data_pair_STC[0])
        
    metasource_STC = ColumnDataSource(data=data_pair_STC[1])
    col = [
        TableColumn(field="Voc", title="Voc"),
        TableColumn(field="Isc", title="Isc"),
        TableColumn(field="Pmax", title="Pmax"),
        TableColumn(field="Vmp", title="Vmp"),
        TableColumn(field="Imp", title="Imp"),
        TableColumn(field="FF", title="Fill Factor"),
    ]
    meta_table_STC = DataTable(source=metasource_STC, columns=col, height=80, index_header='', fit_columns=True)
    

    Temp, Irrad = 40, 600
    source_GT = ColumnDataSource(data=dict(x=[], y=[]))
    metadata_GT = dict(Voc=[], Isc=[], Pmax=[], Vmp=[], Imp=[], FF=[]) 
    metasource_GT = ColumnDataSource(data=metadata_GT)
    meta_table_GT = DataTable(source=metasource_GT, columns=col, height=80, index_header='')
    
    source_corrected = ColumnDataSource(data=dict(x=[], y=[]))    
    metadata_corrected = dict(Voc=[], Isc=[], Pmax=[], Vmp=[], Imp=[], FF=[]) 
    metasource_corrected = ColumnDataSource(data=metadata_corrected)
    meta_table_corrected = DataTable(source=metasource_corrected, columns=col, height=80, index_header='')
    
    plot = Figure(title='I-V plot @STC and given conditions', tools="hover", tooltips="@x: @y", height=500, y_range = (-1, 10), x_range = (0,60))
    plot.xaxis.axis_label = 'Voltage (V)'
    plot.yaxis.axis_label = 'Current (I)'
    plot.scatter('x', 'y', legend='STC', source=source_STC, line_width=3, line_color='green', line_alpha=0.6, name='STC')
    plot.scatter('x', 'y', legend='Translated',source=source_GT, line_width=3,line_color='blue', line_alpha=0.6, name='Translated')
    plot.scatter('x', 'y', legend='Corrected',source=source_corrected, line_width=3,line_color='red', line_alpha=0.6, name='corrected')
    
    # DataSheet Values
    Isc_input = TextInput(value='9.25', title='I_sc', width=150)
    Isc_NOTC_input = TextInput(value='7.47', title='I_sc at NOTC(G=800, T=45C)', width=180)
    Im_input = TextInput(value='8.76', title='I_m', width=150)
    Voc_input = TextInput(value='45.9', title='V_oc', width=150)
    Vm_input = TextInput(value='37.2', title='V_m', width=150)
    Ncell_input = TextInput(value='72', title='No. of cells', width=150)
    alpha_input = TextInput(value='0.005', title='Alpha (Temp. Coeff Isc)', width=150)
    beta_input = TextInput(value='-0.36', title='Beta (Temp. Coeff Voc)', width=150)
    gamma_input = TextInput(value='-0.5', title='Gamma (Temp. Coeff Pmp)', width=160)
    datasize_input = TextInput(value='100', title='Data Size', width=150)
    
    populate_button = Button(label='Populate', button_type='warning',  width=100)
    
    datasheet_row = column(row(Isc_input, Voc_input, Vm_input, Im_input, Ncell_input), 
                           row(alpha_input, beta_input, gamma_input, Isc_NOTC_input, 
                               datasize_input, populate_button))
    
    Irrad_input = TextInput(value='600', title='Irradiance(W/m^2)', width=150)
    Temp_input = TextInput(value='40', title='Temperature (Celcius)', width=150)
    
    # Moudle/Diode params
    Rs_input = TextInput(value='', title='Series Resistance', width=150)
    Rsh_input = TextInput(value='', title='Shunt Resistance', width=150)
    Isat_input = TextInput(value='', title='Isat', width=100)
    Iph_input = TextInput(value='', title='Iph', width=100)
    eta_input = TextInput(value='', title='Eta (Ideality Factor)', width=100)
    submit = Button(label='Submit', button_type='success')
    dwnld = Button(label='Download (csv)',)
    
    
    div_STC = Div(text=""" <b><font color="green">DataTable @ STC</font></b> """)
    div_GT = Div(text=""" <b><font color="blue">DataTable @ given conditions</font></b> """)
    div_corrected = Div(text=""" <b><font color="red">DataTable after STC correction</font></b> """)
    div_param = Div(text=""" <h2><font color="green">5 Module Parameters @STC</font></h2> """)
    div_condition = Div(text=""" <h2><font color="blue">Conditions</font></h2> """)
    div_datasheet = Div(text=""" <h2><font color="blue">Enter DataSheet Values first</font></h2> """)
    div_info = Div(text=""" STC params will get populated after hitting <font color="red"><b>Populate</b></font>""")
    div_note = Div(text=""" <font color="black">Datasheet Parameters are fixed for a given module but overtime the module can get degraded and we might need to change the 5 Module parameters to address this situation</font> """)
    div_note_isc = Div(text=""" <b><font color="red">If you intend to change Iph, then you should do so by changing Isc and I_NOTC values because they are needed for the translation and correction</font></b> """)
    data_div = column(div_datasheet, div_info, datasheet_row, div_note_isc)
    
    def get_input_datasheet():
        return ((float(Voc_input.value) ,float(Isc_input.value) ,float(Vm_input.value) ,float(Im_input.value) ,
                 int(Ncell_input.value)), float(Isc_NOTC_input.value), float(alpha_input.value), float(beta_input.value),
                float(gamma_input.value), int(datasize_input.value))
    
    def get_inputs():
        return (float(Irrad_input.value) ,float(Temp_input.value), 1, 
                (float(Rs_input.value) ,float(Rsh_input.value) ,float(Isat_input.value) ,
                 float(Iph_input.value) ,float(eta_input.value)),)
    
    def update_plot():
        global Temp, Irrad, datasize, datasheet_values, Isc_NOTC, alpha, beta, gamma
        #Taking inputs from the conditions and 5 module parameters fields
        Irrad, Temp, method, module_param = get_inputs()

        #Replotting STC I-V if in case module param is altered
        data_pair_STC = generate_IV(module_param, 25, datasize, datasheet_values[-1])
        source_STC.data = data_pair_STC[0]
        
        # Translating to the given conditions
        params_translated = translate(module_param, Irrad, Temp, Isc_NOTC, datasheet_values[1], alpha)
        data_pair_translated = generate_IV(params_translated, Temp, datasize,  datasheet_values[-1])
        source_GT.data = data_pair_translated[0]            # Updating the translated plot
        metasource_GT.data = data_pair_translated[1]
        
        #Correcting back to STC
        Tcoeff = (alpha, beta, gamma)
        data_corrected = correction(1, data_pair_translated[0], datasheet_values, Irrad, Temp, Tcoeff)
        source_corrected.data = data_corrected[0]
        metasource_corrected.data = data_corrected[1]
    
    
    def populate():
        '''This will fill up automatically the module parameter fields'''
        global Temp, Irrad, datasize, datasheet_values, Isc_NOTC, alpha, beta, gamma
        datasheet_values, Isc_NOTC, alpha, beta, gamma, datasize = get_input_datasheet()

        module_param = generate_5param_STC(datasheet_values)
        

        #Populating the fields
        Isatt = '{:0.2e}'.format(module_param[2])
        module_param_str = [str(np.round(i,2)) for i in module_param]
        module_param_str[2] = Isatt
        Rs_input.value, Rsh_input.value, Isat_input.value, Iph_input.value, eta_input.value = module_param_str
        # Update STC source here too
        #data_pair_STC = generate_IV(module_param, 25, 100, 72)
        data_pair_STC = generate_IV(module_param, 25, datasize, datasheet_values[-1])
        source_STC.data = data_pair_STC[0]
        metasource_STC.data = data_pair_STC[1]
        
        #Resetting translated source and corrected source
        source_GT.data = dict(x=[], y=[])
        source_corrected.data = dict(x=[], y=[])    
        
    def download():
        df = pd.DataFrame(source_STC.data)
        df2 = pd.DataFrame(source_GT.data)
        df3 = pd.DataFrame(source_corrected.data)
        global Temp, Irrad
        df2.to_csv(f'{Irrad}Wpm^2_{Temp}C-ivdata_GT.csv')
        df.to_csv('iv-data_STC.csv')
        df3.to_csv('STC_corrected_iv-data.csv')
    
    populate_button.on_click(populate)
    submit.on_click(update_plot)
    dwnld.on_click(download)
    param_column = column(row(Rs_input, Rsh_input), row(Isat_input, Iph_input, eta_input))
    layout = column(data_div, row(plot, column(div_condition, row(Irrad_input, Temp_input), 
                        column(div_param, param_column), 
                        submit, dwnld),))
    doc.add_root(column(layout, column(div_STC,meta_table_STC), column(div_GT, meta_table_GT), 
                        column(div_corrected, meta_table_corrected)))

show(plot_iv)

Loading BokehJS ...

In [11]:
a = np.arange(1,50,1.5)

In [12]:
a

array([ 1. ,  2.5,  4. ,  5.5,  7. ,  8.5, 10. , 11.5, 13. , 14.5, 16. ,
       17.5, 19. , 20.5, 22. , 23.5, 25. , 26.5, 28. , 29.5, 31. , 32.5,
       34. , 35.5, 37. , 38.5, 40. , 41.5, 43. , 44.5, 46. , 47.5, 49. ])